In [1]:
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
import numpy as np

def get_root_path():
    cwd = os.getcwd()
    while str(os.getcwd()).split('\\')[-1] != 'JealsaML':
        os.chdir("..")
        cwd = os.getcwd()
    return cwd

ROOT_DIR = get_root_path()

sys.path.insert(0, ROOT_DIR)

from src.models.utils import fit_model

# Paths de datos
mypath = os.path.join(ROOT_DIR, 'data\\clean\\Pequeno.csv')         # Origen

# Dataframe principal de Partidas
dtypes = {'Partida':str,
          'Agrupacion':str,
          'MP':str}

dates = ['FechaInicio']
df = pd.read_csv(mypath, dtype=dtypes, parse_dates=dates)
df.columns

Index(['PartidaID', 'Partida', 'FechaInicio', 'Agrupacion', 'ZonaPesca',
       'Proveedor', 'Origen', 'Lomera', 'MP', 'TipoMP', 'NombreMP',
       'EspecieMP', 'TipoCodeMP', 'PequeñoMP', 'VentrescaMP', 'PlantaMP',
       'PesoMedio', 'PesoPartida', 'PesoVersiones', 'MinutosPelado',
       'PesoPales', 'RtoLimpieza', 'MinutosPersonaKg', 'DiasFrigorifico',
       'ZonaFAO', 'Sal', 'Histamina', 'Mercurio', 'Oxidacion', 'Rechazo',
       'Rechazos', 'TipoPesca', 'Carguero'],
      dtype='object')

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
#from keras.layers.core import Dense, Dropout
#from keras.models import Sequential
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

from copy import deepcopy

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Utils para src/model

In [23]:
from random import shuffle

def k_fold_cross_validation(items, k, randomize=False):

    if randomize:
        items = list(items)
        shuffle(items)

    slices = [items[i::k] for i in range(k)]

    for i in range(k):
        validation = slices[i]
        training = [item
                    for s in slices if s is not validation
                    for item in s]
        yield training, validation
        

def split_data_by_index(df, train_idx, test_idx, target):
    """
    A partir de un df divide obtiene las matrices numpy
    X_train, X_test, y_train, y_test segun unos indices y 
    una variable target
    """
    # Preparamos los datos de entrada en los modelos
    df_train, df_test = data_ml.ix[train_idx], data_ml.ix[test_idx]
        
    # Convertimos features y target en numpies
    X_train = np.array(df_train.drop(columns=[target], axis= 1))
    X_test = np.array(df_test.drop(columns=[target], axis= 1))
    y_train = np.array(df_train[target])
    y_test = np.array(df_test[target])
    
    return X_train, X_test, y_train, y_test
    

def fit_model(model, X_train, y_train, X_test, y_test):
    """
    Fita un modelo de regresion a partir de unos datos:"""
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    err_train = get_error(y_train, model.predict(X_train))
    err_test = get_error(y_test, y_pred)

    return model, y_pred, err_train, err_test


def get_error(y_real, y_pred):
    mae = mean_absolute_error(y_real, y_pred)
    rmse = np.sqrt(mean_squared_error(y_real, y_pred))
    r2 = r2_score(y_real, y_pred)
    
    return [mae, rmse, r2]

# Modelo 1 - Variables antiguas

In [4]:
# PREPROCESSING

## ROW SELECTION
def drop_missings_pequeño(df):
    df = df.loc[~df.PesoPales.isna()]
    df = df.loc[df.RtoLimpieza > 0.3]
    
    return df

df = drop_missings_pequeño(df)

# Eliminamos datos del PAC
df = df.loc[df.ZonaPesca != 'Pacifico']

df = df.reset_index(drop=True)

## FEATURE SELECTION
data_ml = df.loc[:,['ZonaPesca','EspecieMP','TipoPesca', 'DiasFrigorifico',
                    'Sal','Histamina','PesoMedio','MinutosPersonaKg','RtoLimpieza']]




## MISSING IMPUTATION
data_ml = data_ml.fillna(0)

## CATEGORICAL ENCODING
data_ml = pd.get_dummies(data_ml).fillna(0)

data_ml.describe()

,DiasFrigorifico,Sal,Histamina,PesoMedio,MinutosPersonaKg,RtoLimpieza,ZonaPesca_Atlantico,ZonaPesca_Indico,EspecieMP_BI,EspecieMP_SK,EspecieMP_YF,TipoPesca_FAD_FREE,TipoPesca_LINE_MSC,TipoPesca_MSC,TipoPesca_POLE_LINE,TipoPesca_STANDARD
count,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000
mean,89.152331,0.848361,0.711085,3.073936,1.592539,40.712205,0.358108,0.641892,0.180743,0.535473,0.283784,0.015203,0.027027,0.006757,0.111486,0.839527
std,72.226622,0.480656,4.070795,1.079419,0.386625,3.261236,0.479850,0.479850,0.385131,0.499162,0.451214,0.122462,0.162299,0.081991,0.315000,0.367355
min,1.000000,0.000000,0.000000,1.080000,0.307582,2.850972,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34.000000,0.550556,0.000000,2.460000,1.335466,39.551822,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,70.000000,0.912500,0.000000,2.860000,1.504687,41.022675,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,119.500000,1.123929,0.000000,3.380000,1.793675,42.314347,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,359.000000,3.217500,56.946391,9.390000,2.957214,49.254630,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
# MODELS

# Definimos o scaler e os modelos
scaler = ('scaler', MinMaxScaler())

models = [('Ridge', ('RDG', Ridge(normalize=True))),
          ('Lasso', ('LSS', Lasso()))]

In [27]:
# FITTING

# Cogemos los nombres de las variables para usarlos más adelante
features_list = data_ml.columns
target = 'RtoLimpieza'


def compare_regression_models_cv(df, models, target, scaler, k=10):
    """
    Compara modelos de regresion por Cross Validation
    
    Devuelve un dataframe con los resultados MAE, RMSE y R2 por modelo y set (train/test)
    """
    
    # Creamos la lista de items para poder seleccionar del indice
    items = [i for i in range(df.shape[0])]

    results = []

    for train_idx, test_idx in k_fold_cross_validation(items, k=k, randomize=True):

        X_train, X_test, y_train, y_test = split_data_by_index(df, train_idx, test_idx, target)

        for name, model in models:
            pipeline = Pipeline(steps =[scaler,model])

            # FITAMOS
            _, _, err_train, err_test = fit_model(pipeline, X_train, y_train, X_test, y_test)

            # CARGAMOS RESULTADOS
            results.append([name, "train"] + err_train)
            results.append([name, "test"] + err_test)
            
    return pd.DataFrame(data = results, columns = ['Model', 'Set', 'MAE','RMSE','R2'])
        
compare_regression_models_cv(data_ml, models, target, scaler, k=10)

,Model,Set,MAE,RMSE,R2
0,Ridge,train,1.611077,2.966217,1.210496e-01
1,Ridge,test,1.829895,3.840129,6.892614e-02
2,Lasso,train,1.818424,3.163885,0.000000e+00
3,Lasso,test,2.031028,4.002898,-1.167626e-02
4,Ridge,train,1.690258,3.161309,1.195944e-01
5,Ridge,test,1.379943,1.889467,8.837794e-02
6,Lasso,train,1.892455,3.369191,0.000000e+00
7,Lasso,test,1.502352,2.033190,-5.558257e-02
8,Ridge,train,1.587387,2.742703,1.417751e-01
9,Ridge,test,2.053858,5.135406,2.535301e-02


In [25]:
results = 
results.head()


,Model,Set,MAE,RMSE,R2
0,Ridge,train,1.644993,3.122968,0.125388
1,Ridge,test,1.755407,2.384857,-0.028441
2,Lasso,train,1.851283,3.339334,0.000000
3,Lasso,test,1.893243,2.434762,-0.071934
4,Ridge,train,1.704967,3.188181,0.114483


In [34]:
results = {}
results_train ={}
modelos = ['manual', 'ridge','lasso','cart','svr','xgb','rf', 'ann']
for modelo in modelos:
    results[modelo]=[]
    results_train[modelo]=[]


items = [i for i in range(data_ml.shape[0])]
for training, validation in k_fold_cross_validation(items, 10):
    
    # Preparamos los datos de entrada en los modelos
    df_train = data_ml.ix[training]
    df_validation = data_ml.ix[validation]
    
    #######################################################
    ###### DATOS PARA ML
    #######################################################
    
    df_train_dummies = pd.get_dummies(df_train).fillna(0)
    df_validation_dummies = pd.get_dummies(df_validation).fillna(0)
    
    # checkear que tengan las mismas columnas
    for column in df_train_dummies.columns:
        if column not in df_validation_dummies:
            df_validation_dummies[column]=0
    
    features_list = df_train_dummies.columns
    
    # Convertimos features y target en numpies
    X_train = np.array(df_train_dummies.drop(columns=['RtoLimpieza'], axis= 1))
    X_test = np.array(df_validation_dummies.drop(columns=['RtoLimpieza'], axis= 1))
    y_train = np.array(df_train_dummies['RtoLimpieza'])
    y_test = np.array(df_validation_dummies['RtoLimpieza'])
    
    # Normalizamos los predictores
    Xscaler = MinMaxScaler()
    Xscaler.fit(X_train)
    X_train = Xscaler.transform(X_train)
    X_test = Xscaler.transform(X_test)
    
    #######################################################
    ###### RIDGE REGRESSION
    #######################################################
    
    ridge = Ridge(normalize=True)
    ridge.fit(X_train, y_train)
    y_pred_ridge = ridge.predict(X_test)
    results['ridge'].append(get_error(y_test, y_pred_ridge))
    results_train['ridge'].append(get_error(y_train, ridge.predict(X_train)))
    

    
    
    #######################################################
    ###### LASSO REGRESSION
    #######################################################
    
    lasso = Lasso()
    lasso.fit(X_train, y_train)
    y_pred_lasso = lasso.predict(X_test)
    results['lasso'].append(get_error(y_test, y_pred_lasso))
    results_train['lasso'].append(get_error(y_train, lasso.predict(X_train)))
    
    
    #######################################################
    ###### CART
    #######################################################
    
    cart = DecisionTreeRegressor()
    cart.fit(X_train, y_train)
    y_pred_cart = cart.predict(X_test)
    results['cart'].append(get_error(y_test, y_pred_cart))
    results_train['cart'].append(get_error(y_train, cart.predict(X_train)))
    
    #######################################################
    ###### SVC REGRESOR
    #######################################################
    
    svr = SVR()
    svr.fit(X_train, y_train)
    y_pred_svr = svr.predict(X_test)
    results['svr'].append(get_error(y_test, y_pred_svr))
    results_train['svr'].append(get_error(y_train, svr.predict(X_train)))
    
    #######################################################
    ###### XGB
    #######################################################
    
    xgb = GradientBoostingRegressor()
    xgb.fit(X_train, y_train)
    y_pred_xgb = xgb.predict(X_test)
    results['xgb'].append(get_error(y_test, y_pred_xgb))
    results_train['xgb'].append(get_error(y_train, xgb.predict(X_train)))
    
    #######################################################
    ###### RANDOM FOREST
    #######################################################

    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)
    results['rf'].append(get_error(y_test, y_pred_rf))
    results_train['rf'].append(get_error(y_train, rf.predict(X_train)))
    
    #######################################################
    ###### DATA PREP FOR ANN
    #######################################################
    """
    yscaler = MinMaxScaler()
    yscaler.fit(y_train.reshape(-1,1))
    y_train_ann = yscaler.transform(y_train.reshape(-1,1))
    y_test_ann = yscaler.transform(y_test.reshape(-1,1))
    
    #######################################################
    ###### ANN - MODEL CONSTRUCTION
    #######################################################
    
    annmodel = Sequential()
    annmodel.add(Dense(50, activation='relu', input_shape=(X_train.shape[1],)))
    annmodel.add(Dropout(0.2))
    annmodel.add(Dense(25, activation='relu'))
    annmodel.add(Dense(1, activation = "sigmoid"))
    annmodel.compile(loss='mse', optimizer='adam')
    
    #######################################################
    ###### ANN
    #######################################################
    
    # Entrenamos el modelo
    annmodel.fit(X_train, y_train_ann, epochs=10, verbose=0, 
          validation_data = (X_test, y_test_ann))
    y_pred_ann = annmodel.predict(X_test)
    y_pred_ann_desnorm = yscaler.inverse_transform(y_pred_ann)
    results['ann'].append(get_error(y_test, y_pred_ann_desnorm))
    
    y_pred_ann_train = yscaler.inverse_transform(annmodel.predict(X_train))
    results_train['ann'].append(get_error(y_train, y_pred_ann_train))
    """

MinMaxScaler(copy=True, feature_range=(0, 1))

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

...

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

'\n    yscaler = MinMaxScaler()\n    yscaler.fit(y_train.reshape(-1,1))\n    y_train_ann = yscaler.transform(y_train.reshape(-1,1))\n    y_test_ann = yscaler.transform(y_test.reshape(-1,1))\n    \n    #######################################################\n    ###### ANN - MODEL CONSTRUCTION\n    #######################################################\n    \n    annmodel = Sequential()\n    annmodel.add(Dense(50, activation=\'relu\', input_shape=(X_train.shape[1],)))\n    annmodel.add(Dropout(0.2))\n    annmodel.add(Dense(25, activation=\'relu\'))\n    annmodel.add(Dense(1, activation = "sigmoid"))\n    annmodel.compile(loss=\'mse\', optimizer=\'adam\')\n    \n    #######################################################\n    ###### ANN\n    #######################################################\n    \n    # Entrenamos el modelo\n    annmodel.fit(X_train, y_train_ann, epochs=10, verbose=0, \n          validation_data = (X_test, y_test_ann))\n    y_pred_ann = annmodel.predict(X_test)\n 

## Resultados 

In [35]:
df_results =[]
for key in results.keys():
    result = pd.DataFrame(data=results[key], columns=['mae','rmse','r2'])
    df_results.append([key, result.mae.mean(),result.mae.std(),
                      result.rmse.mean(),result.rmse.std(),
                    result.r2.mean(),result.r2.std()])
    
df_results = pd.DataFrame(data=df_results, columns = ['modelo','mae_mean', 'mae_std',
                                                      'rmse_mean','rmse_std', 'r2_mean','r2_std'])

df_results

,modelo,mae_mean,mae_std,rmse_mean,rmse_std,r2_mean,r2_std
0,manual,NaN,NaN,NaN,NaN,NaN,NaN
1,ridge,3.132913,0.546874,4.894802,1.119125,0.216245,0.512191
2,lasso,2.781429,0.583474,6.162906,2.047244,-0.018191,0.025826
3,cart,1.810579,0.211867,2.654796,0.430105,0.609650,0.623526
4,svr,2.379359,0.538384,5.900599,2.038506,0.079525,0.036457
5,xgb,1.384249,0.194915,1.965040,0.450217,0.798501,0.270216
6,rf,1.356447,0.142933,1.936297,0.354708,0.811613,0.246810
7,ann,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_results =[]
for key in results_train.keys():
    result = pd.DataFrame(data=results_train[key], columns=['mae','rmse','r2'])
    df_results.append([key, result.mae.mean(),result.mae.std(),
                      result.rmse.mean(),result.rmse.std(),
                    result.r2.mean(),result.r2.std()])
    
df_results = pd.DataFrame(data=df_results, columns = ['modelo','mae_mean', 'mae_std',
                                                      'rmse_mean','rmse_std', 'r2_mean','r2_std'])

df_results

,modelo,mae_mean,mae_std,rmse_mean,rmse_std,r2_mean,r2_std
0,manual,NaN,NaN,NaN,NaN,NaN,NaN
1,ridge,2.519981e+00,5.748583e-02,4.635986,0.063348,0.482441,2.281545e-02
2,lasso,2.780842e+00,8.990913e-02,6.449194,0.208345,0.000071,2.245328e-04
3,cart,3.386065e-07,3.617094e-07,0.000005,0.000005,1.000000,1.626572e-12
4,svr,2.298805e+00,5.372440e-02,6.172554,0.175214,0.083781,8.716088e-03
5,xgb,9.064189e-01,1.120850e-02,1.190058,0.016266,0.965870,2.097016e-03
6,rf,5.407604e-01,2.358529e-02,0.876697,0.116868,0.981051,6.039177e-03
7,ann,NaN,NaN,NaN,NaN,NaN,NaN


# Modelo 2 - Variable Rechazo, Oxidacion y Origen-Contenedor

In [37]:
data_ml = df.loc[:,['ZonaPesca','EspecieMP','ModalidadLimpieza','TipoPesca', 'DiasFrigorifico',
                              'Sal','Histamina',
          'PesoMedio','MinutosPersonaKg','RtoLimpieza', 'Rechazos','Oxidacion','Carguero']]

data_ml.shape

(592, 13)

In [38]:
results = {}
results_train ={}
modelos = ['manual', 'ridge','lasso','cart','svr','xgb','rf', 'ann']
for modelo in modelos:
    results[modelo]=[]
    results_train[modelo]=[]


items = [i for i in range(data_ml.shape[0])]
for training, validation in k_fold_cross_validation(items, 10):
    
    # Preparamos los datos de entrada en los modelos
    df_train = data_ml.ix[training]
    df_validation = data_ml.ix[validation]
    
    #######################################################
    ###### DATOS PARA ML
    #######################################################
    
    df_train_dummies = pd.get_dummies(df_train).fillna(0)
    df_validation_dummies = pd.get_dummies(df_validation).fillna(0)
    
    # checkear que tengan las mismas columnas
    for column in df_train_dummies.columns:
        if column not in df_validation_dummies:
            df_validation_dummies[column]=0
    
    features_list = df_train_dummies.columns
    
    # Convertimos features y target en numpies
    X_train = np.array(df_train_dummies.drop(columns=['RtoLimpieza'], axis= 1))
    X_test = np.array(df_validation_dummies.drop(columns=['RtoLimpieza'], axis= 1))
    y_train = np.array(df_train_dummies['RtoLimpieza'])
    y_test = np.array(df_validation_dummies['RtoLimpieza'])
    
    # Normalizamos los predictores
    Xscaler = MinMaxScaler()
    Xscaler.fit(X_train)
    X_train = Xscaler.transform(X_train)
    X_test = Xscaler.transform(X_test)
    
    #######################################################
    ###### RIDGE REGRESSION
    #######################################################
    
    ridge = Ridge(normalize=True)
    ridge.fit(X_train, y_train)
    y_pred_ridge = ridge.predict(X_test)
    results['ridge'].append(get_error(y_test, y_pred_ridge))
    results_train['ridge'].append(get_error(y_train, ridge.predict(X_train)))
    
    
    #######################################################
    ###### LASSO REGRESSION
    #######################################################
    
    lasso = Lasso()
    lasso.fit(X_train, y_train)
    y_pred_lasso = lasso.predict(X_test)
    results['lasso'].append(get_error(y_test, y_pred_lasso))
    results_train['lasso'].append(get_error(y_train, lasso.predict(X_train)))
    
    
    #######################################################
    ###### CART
    #######################################################
    
    cart = DecisionTreeRegressor()
    cart.fit(X_train, y_train)
    y_pred_cart = cart.predict(X_test)
    results['cart'].append(get_error(y_test, y_pred_cart))
    results_train['cart'].append(get_error(y_train, cart.predict(X_train)))
    
    #######################################################
    ###### SVC REGRESOR
    #######################################################
    
    svr = SVR()
    svr.fit(X_train, y_train)
    y_pred_svr = svr.predict(X_test)
    results['svr'].append(get_error(y_test, y_pred_svr))
    results_train['svr'].append(get_error(y_train, svr.predict(X_train)))
    
    #######################################################
    ###### XGB
    #######################################################
    
    xgb = GradientBoostingRegressor()
    xgb.fit(X_train, y_train)
    y_pred_xgb = xgb.predict(X_test)
    results['xgb'].append(get_error(y_test, y_pred_xgb))
    results_train['xgb'].append(get_error(y_train, xgb.predict(X_train)))
    
    #######################################################
    ###### RANDOM FOREST
    #######################################################

    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)
    results['rf'].append(get_error(y_test, y_pred_rf))
    results_train['rf'].append(get_error(y_train, rf.predict(X_train)))
    
    #######################################################
    ###### DATA PREP FOR ANN
    #######################################################
    """
    yscaler = MinMaxScaler()
    yscaler.fit(y_train.reshape(-1,1))
    y_train_ann = yscaler.transform(y_train.reshape(-1,1))
    y_test_ann = yscaler.transform(y_test.reshape(-1,1))
    
    #######################################################
    ###### ANN - MODEL CONSTRUCTION
    #######################################################
    
    annmodel = Sequential()
    annmodel.add(Dense(50, activation='relu', input_shape=(X_train.shape[1],)))
    annmodel.add(Dropout(0.2))
    annmodel.add(Dense(25, activation='relu'))
    annmodel.add(Dense(1, activation = "sigmoid"))
    annmodel.compile(loss='mse', optimizer='adam')
    
    #######################################################
    ###### ANN
    #######################################################
    
    # Entrenamos el modelo
    annmodel.fit(X_train, y_train_ann, epochs=10, verbose=0, 
          validation_data = (X_test, y_test_ann))
    y_pred_ann = annmodel.predict(X_test)
    y_pred_ann_desnorm = yscaler.inverse_transform(y_pred_ann)
    results['ann'].append(get_error(y_test, y_pred_ann_desnorm))
    
    y_pred_ann_train = yscaler.inverse_transform(annmodel.predict(X_train))
    results_train['ann'].append(get_error(y_train, y_pred_ann_train))
    """

MinMaxScaler(copy=True, feature_range=(0, 1))

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

...

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

'\n    yscaler = MinMaxScaler()\n    yscaler.fit(y_train.reshape(-1,1))\n    y_train_ann = yscaler.transform(y_train.reshape(-1,1))\n    y_test_ann = yscaler.transform(y_test.reshape(-1,1))\n    \n    #######################################################\n    ###### ANN - MODEL CONSTRUCTION\n    #######################################################\n    \n    annmodel = Sequential()\n    annmodel.add(Dense(50, activation=\'relu\', input_shape=(X_train.shape[1],)))\n    annmodel.add(Dropout(0.2))\n    annmodel.add(Dense(25, activation=\'relu\'))\n    annmodel.add(Dense(1, activation = "sigmoid"))\n    annmodel.compile(loss=\'mse\', optimizer=\'adam\')\n    \n    #######################################################\n    ###### ANN\n    #######################################################\n    \n    # Entrenamos el modelo\n    annmodel.fit(X_train, y_train_ann, epochs=10, verbose=0, \n          validation_data = (X_test, y_test_ann))\n    y_pred_ann = annmodel.predict(X_test)\n 

## Resultados

In [39]:
df_results =[]
for key in results.keys():
    result = pd.DataFrame(data=results[key], columns=['mae','rmse','r2'])
    df_results.append([key, result.mae.mean(),result.mae.std(),
                      result.rmse.mean(),result.rmse.std(),
                    result.r2.mean(),result.r2.std()])
    
df_results = pd.DataFrame(data=df_results, columns = ['modelo','mae_mean', 'mae_std',
                                                      'rmse_mean','rmse_std', 'r2_mean','r2_std'])

df_results

,modelo,mae_mean,mae_std,rmse_mean,rmse_std,r2_mean,r2_std
0,manual,NaN,NaN,NaN,NaN,NaN,NaN
1,ridge,3.078949,0.595590,4.809704,1.129568,0.242801,0.504656
2,lasso,2.781482,0.583450,6.162875,2.047277,-0.018176,0.025811
3,cart,1.413857,0.294046,2.239973,0.815805,0.653367,0.569762
4,svr,2.292800,0.564400,5.904009,2.097801,0.086386,0.055709
5,xgb,1.056555,0.129669,1.608047,0.383889,0.857028,0.208642
6,rf,1.037363,0.172697,1.708259,0.776506,0.818834,0.285992
7,ann,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df_results =[]
for key in results_train.keys():
    result = pd.DataFrame(data=results_train[key], columns=['mae','rmse','r2'])
    df_results.append([key, result.mae.mean(),result.mae.std(),
                      result.rmse.mean(),result.rmse.std(),
                    result.r2.mean(),result.r2.std()])
    
df_results = pd.DataFrame(data=df_results, columns = ['modelo','mae_mean', 'mae_std',
                                                      'rmse_mean','rmse_std', 'r2_mean','r2_std'])

df_results

,modelo,mae_mean,mae_std,rmse_mean,rmse_std,r2_mean,r2_std
0,manual,NaN,NaN,NaN,NaN,NaN,NaN
1,ridge,2.394885,5.925209e-02,4.511971,0.059540,0.509761,2.145682e-02
2,lasso,2.780842,8.990913e-02,6.449194,0.208345,0.000071,2.245328e-04
3,cart,0.000001,7.494237e-07,0.000013,0.000006,1.000000,3.924728e-12
4,svr,2.186805,5.577380e-02,6.191210,0.188084,0.078356,5.339998e-03
5,xgb,0.643618,7.793197e-03,0.884816,0.014282,0.981108,1.532863e-03
6,rf,0.405159,1.381396e-02,0.669441,0.040359,0.989194,1.165032e-03
7,ann,NaN,NaN,NaN,NaN,NaN,NaN


# Modelo 3 - Variable Rechazo

In [41]:
data_ml = df.loc[:,['ZonaPesca','EspecieMP','ModalidadLimpieza','TipoPesca', 'DiasFrigorifico',
                              'Sal','Histamina',
          'PesoMedio','MinutosPersonaKg','RtoLimpieza', 'Rechazos']]

data_ml.shape

(592, 11)

In [42]:
results = {}
results_train ={}
modelos = ['manual', 'ridge','lasso','cart','svr','xgb','rf', 'ann']
for modelo in modelos:
    results[modelo]=[]
    results_train[modelo]=[]


items = [i for i in range(data_ml.shape[0])]
for training, validation in k_fold_cross_validation(items, 10):
    
    # Preparamos los datos de entrada en los modelos
    df_train = data_ml.ix[training]
    df_validation = data_ml.ix[validation]
    
    #######################################################
    ###### DATOS PARA ML
    #######################################################
    
    df_train_dummies = pd.get_dummies(df_train).fillna(0)
    df_validation_dummies = pd.get_dummies(df_validation).fillna(0)
    
    # checkear que tengan las mismas columnas
    for column in df_train_dummies.columns:
        if column not in df_validation_dummies:
            df_validation_dummies[column]=0
    
    features_list = df_train_dummies.columns
    
    # Convertimos features y target en numpies
    X_train = np.array(df_train_dummies.drop(columns=['RtoLimpieza'], axis= 1))
    X_test = np.array(df_validation_dummies.drop(columns=['RtoLimpieza'], axis= 1))
    y_train = np.array(df_train_dummies['RtoLimpieza'])
    y_test = np.array(df_validation_dummies['RtoLimpieza'])
    
    # Normalizamos los predictores
    Xscaler = MinMaxScaler()
    Xscaler.fit(X_train)
    X_train = Xscaler.transform(X_train)
    X_test = Xscaler.transform(X_test)
    
    #######################################################
    ###### RIDGE REGRESSION
    #######################################################
    
    ridge = Ridge(normalize=True)
    ridge.fit(X_train, y_train)
    y_pred_ridge = ridge.predict(X_test)
    results['ridge'].append(get_error(y_test, y_pred_ridge))
    results_train['ridge'].append(get_error(y_train, ridge.predict(X_train)))
    
    
    #######################################################
    ###### LASSO REGRESSION
    #######################################################
    
    lasso = Lasso()
    lasso.fit(X_train, y_train)
    y_pred_lasso = lasso.predict(X_test)
    results['lasso'].append(get_error(y_test, y_pred_lasso))
    results_train['lasso'].append(get_error(y_train, lasso.predict(X_train)))
    
    
    #######################################################
    ###### CART
    #######################################################
    
    cart = DecisionTreeRegressor()
    cart.fit(X_train, y_train)
    y_pred_cart = cart.predict(X_test)
    results['cart'].append(get_error(y_test, y_pred_cart))
    results_train['cart'].append(get_error(y_train, cart.predict(X_train)))
    
    #######################################################
    ###### SVC REGRESOR
    #######################################################
    
    svr = SVR()
    svr.fit(X_train, y_train)
    y_pred_svr = svr.predict(X_test)
    results['svr'].append(get_error(y_test, y_pred_svr))
    results_train['svr'].append(get_error(y_train, svr.predict(X_train)))
    
    #######################################################
    ###### XGB
    #######################################################
    
    xgb = GradientBoostingRegressor()
    xgb.fit(X_train, y_train)
    y_pred_xgb = xgb.predict(X_test)
    results['xgb'].append(get_error(y_test, y_pred_xgb))
    results_train['xgb'].append(get_error(y_train, xgb.predict(X_train)))
    
    #######################################################
    ###### RANDOM FOREST
    #######################################################

    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)
    results['rf'].append(get_error(y_test, y_pred_rf))
    results_train['rf'].append(get_error(y_train, rf.predict(X_train)))
    
    #######################################################
    ###### DATA PREP FOR ANN
    #######################################################
    """
    yscaler = MinMaxScaler()
    yscaler.fit(y_train.reshape(-1,1))
    y_train_ann = yscaler.transform(y_train.reshape(-1,1))
    y_test_ann = yscaler.transform(y_test.reshape(-1,1))
    
    #######################################################
    ###### ANN - MODEL CONSTRUCTION
    #######################################################
    
    annmodel = Sequential()
    annmodel.add(Dense(50, activation='relu', input_shape=(X_train.shape[1],)))
    annmodel.add(Dropout(0.2))
    annmodel.add(Dense(25, activation='relu'))
    annmodel.add(Dense(1, activation = "sigmoid"))
    annmodel.compile(loss='mse', optimizer='adam')
    
    #######################################################
    ###### ANN
    #######################################################
    
    # Entrenamos el modelo
    annmodel.fit(X_train, y_train_ann, epochs=10, verbose=0, 
          validation_data = (X_test, y_test_ann))
    y_pred_ann = annmodel.predict(X_test)
    y_pred_ann_desnorm = yscaler.inverse_transform(y_pred_ann)
    results['ann'].append(get_error(y_test, y_pred_ann_desnorm))
    
    y_pred_ann_train = yscaler.inverse_transform(annmodel.predict(X_train))
    results_train['ann'].append(get_error(y_train, y_pred_ann_train))
    """

MinMaxScaler(copy=True, feature_range=(0, 1))

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

...

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

'\n    yscaler = MinMaxScaler()\n    yscaler.fit(y_train.reshape(-1,1))\n    y_train_ann = yscaler.transform(y_train.reshape(-1,1))\n    y_test_ann = yscaler.transform(y_test.reshape(-1,1))\n    \n    #######################################################\n    ###### ANN - MODEL CONSTRUCTION\n    #######################################################\n    \n    annmodel = Sequential()\n    annmodel.add(Dense(50, activation=\'relu\', input_shape=(X_train.shape[1],)))\n    annmodel.add(Dropout(0.2))\n    annmodel.add(Dense(25, activation=\'relu\'))\n    annmodel.add(Dense(1, activation = "sigmoid"))\n    annmodel.compile(loss=\'mse\', optimizer=\'adam\')\n    \n    #######################################################\n    ###### ANN\n    #######################################################\n    \n    # Entrenamos el modelo\n    annmodel.fit(X_train, y_train_ann, epochs=10, verbose=0, \n          validation_data = (X_test, y_test_ann))\n    y_pred_ann = annmodel.predict(X_test)\n 

## Resultados

In [43]:
df_results =[]
for key in results.keys():
    result = pd.DataFrame(data=results[key], columns=['mae','rmse','r2'])
    df_results.append([key, result.mae.mean(),result.mae.std(),
                      result.rmse.mean(),result.rmse.std(),
                    result.r2.mean(),result.r2.std()])
    
df_results = pd.DataFrame(data=df_results, columns = ['modelo','mae_mean', 'mae_std',
                                                      'rmse_mean','rmse_std', 'r2_mean','r2_std'])

df_results

,modelo,mae_mean,mae_std,rmse_mean,rmse_std,r2_mean,r2_std
0,manual,NaN,NaN,NaN,NaN,NaN,NaN
1,ridge,3.085272,0.572981,4.865852,1.136027,0.225035,0.515967
2,lasso,2.781429,0.583474,6.162906,2.047244,-0.018191,0.025826
3,cart,1.451042,0.319605,2.268084,0.837359,0.617393,0.671338
4,svr,2.273794,0.554045,5.897361,2.084910,0.085880,0.046455
5,xgb,1.015021,0.127444,1.528596,0.322364,0.870502,0.195830
6,rf,1.037854,0.138027,1.607035,0.513944,0.852290,0.212720
7,ann,NaN,NaN,NaN,NaN,NaN,NaN


# Variable Oxidacion

In [44]:
data_ml = df.loc[:,['ZonaPesca','EspecieMP','ModalidadLimpieza','TipoPesca', 'DiasFrigorifico',
                              'Sal','Histamina',
          'PesoMedio','MinutosPersonaKg','RtoLimpieza', 'Oxidacion']]

data_ml.shape

(592, 11)

In [45]:
results = {}
results_train ={}
modelos = ['manual', 'ridge','lasso','cart','svr','xgb','rf', 'ann']
for modelo in modelos:
    results[modelo]=[]
    results_train[modelo]=[]


items = [i for i in range(data_ml.shape[0])]
for training, validation in k_fold_cross_validation(items, 10):
    
    # Preparamos los datos de entrada en los modelos
    df_train = data_ml.ix[training]
    df_validation = data_ml.ix[validation]
    
    #######################################################
    ###### DATOS PARA ML
    #######################################################
    
    df_train_dummies = pd.get_dummies(df_train).fillna(0)
    df_validation_dummies = pd.get_dummies(df_validation).fillna(0)
    
    # checkear que tengan las mismas columnas
    for column in df_train_dummies.columns:
        if column not in df_validation_dummies:
            df_validation_dummies[column]=0
    
    features_list = df_train_dummies.columns
    
    # Convertimos features y target en numpies
    X_train = np.array(df_train_dummies.drop(columns=['RtoLimpieza'], axis= 1))
    X_test = np.array(df_validation_dummies.drop(columns=['RtoLimpieza'], axis= 1))
    y_train = np.array(df_train_dummies['RtoLimpieza'])
    y_test = np.array(df_validation_dummies['RtoLimpieza'])
    
    # Normalizamos los predictores
    Xscaler = MinMaxScaler()
    Xscaler.fit(X_train)
    X_train = Xscaler.transform(X_train)
    X_test = Xscaler.transform(X_test)
    
    #######################################################
    ###### RIDGE REGRESSION
    #######################################################
    
    ridge = Ridge(normalize=True)
    ridge.fit(X_train, y_train)
    y_pred_ridge = ridge.predict(X_test)
    results['ridge'].append(get_error(y_test, y_pred_ridge))
    results_train['ridge'].append(get_error(y_train, ridge.predict(X_train)))
    
    def fit_model(model):
        
    
    
    #######################################################
    ###### LASSO REGRESSION
    #######################################################
    
    lasso = Lasso()
    lasso.fit(X_train, y_train)
    y_pred_lasso = lasso.predict(X_test)
    results['lasso'].append(get_error(y_test, y_pred_lasso))
    results_train['lasso'].append(get_error(y_train, lasso.predict(X_train)))
    
    
    #######################################################
    ###### CART
    #######################################################
    
    cart = DecisionTreeRegressor()
    cart.fit(X_train, y_train)
    y_pred_cart = cart.predict(X_test)
    results['cart'].append(get_error(y_test, y_pred_cart))
    results_train['cart'].append(get_error(y_train, cart.predict(X_train)))
    
    #######################################################
    ###### SVC REGRESOR
    #######################################################
    
    svr = SVR()
    svr.fit(X_train, y_train)
    y_pred_svr = svr.predict(X_test)
    results['svr'].append(get_error(y_test, y_pred_svr))
    results_train['svr'].append(get_error(y_train, svr.predict(X_train)))
    
    #######################################################
    ###### XGB
    #######################################################
    
    xgb = GradientBoostingRegressor()
    xgb.fit(X_train, y_train)
    y_pred_xgb = xgb.predict(X_test)
    results['xgb'].append(get_error(y_test, y_pred_xgb))
    results_train['xgb'].append(get_error(y_train, xgb.predict(X_train)))
    
    #######################################################
    ###### RANDOM FOREST
    #######################################################

    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)
    results['rf'].append(get_error(y_test, y_pred_rf))
    results_train['rf'].append(get_error(y_train, rf.predict(X_train)))
    
    #######################################################
    ###### DATA PREP FOR ANN
    #######################################################
    """
    yscaler = MinMaxScaler()
    yscaler.fit(y_train.reshape(-1,1))
    y_train_ann = yscaler.transform(y_train.reshape(-1,1))
    y_test_ann = yscaler.transform(y_test.reshape(-1,1))
    
    #######################################################
    ###### ANN - MODEL CONSTRUCTION
    #######################################################
    
    annmodel = Sequential()
    annmodel.add(Dense(50, activation='relu', input_shape=(X_train.shape[1],)))
    annmodel.add(Dropout(0.2))
    annmodel.add(Dense(25, activation='relu'))
    annmodel.add(Dense(1, activation = "sigmoid"))
    annmodel.compile(loss='mse', optimizer='adam')
    
    #######################################################
    ###### ANN
    #######################################################
    
    # Entrenamos el modelo
    annmodel.fit(X_train, y_train_ann, epochs=10, verbose=0, 
          validation_data = (X_test, y_test_ann))
    y_pred_ann = annmodel.predict(X_test)
    y_pred_ann_desnorm = yscaler.inverse_transform(y_pred_ann)
    results['ann'].append(get_error(y_test, y_pred_ann_desnorm))
    
    y_pred_ann_train = yscaler.inverse_transform(annmodel.predict(X_train))
    results_train['ann'].append(get_error(y_train, y_pred_ann_train))
    """

MinMaxScaler(copy=True, feature_range=(0, 1))

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

...

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

'\n    yscaler = MinMaxScaler()\n    yscaler.fit(y_train.reshape(-1,1))\n    y_train_ann = yscaler.transform(y_train.reshape(-1,1))\n    y_test_ann = yscaler.transform(y_test.reshape(-1,1))\n    \n    #######################################################\n    ###### ANN - MODEL CONSTRUCTION\n    #######################################################\n    \n    annmodel = Sequential()\n    annmodel.add(Dense(50, activation=\'relu\', input_shape=(X_train.shape[1],)))\n    annmodel.add(Dropout(0.2))\n    annmodel.add(Dense(25, activation=\'relu\'))\n    annmodel.add(Dense(1, activation = "sigmoid"))\n    annmodel.compile(loss=\'mse\', optimizer=\'adam\')\n    \n    #######################################################\n    ###### ANN\n    #######################################################\n    \n    # Entrenamos el modelo\n    annmodel.fit(X_train, y_train_ann, epochs=10, verbose=0, \n          validation_data = (X_test, y_test_ann))\n    y_pred_ann = annmodel.predict(X_test)\n 

## Resultados

In [46]:
df_results =[]
for key in results.keys():
    result = pd.DataFrame(data=results[key], columns=['mae','rmse','r2'])
    df_results.append([key, result.mae.mean(),result.mae.std(),
                      result.rmse.mean(),result.rmse.std(),
                    result.r2.mean(),result.r2.std()])
    
df_results = pd.DataFrame(data=df_results, columns = ['modelo','mae_mean', 'mae_std',
                                                      'rmse_mean','rmse_std', 'r2_mean','r2_std'])

df_results

,modelo,mae_mean,mae_std,rmse_mean,rmse_std,r2_mean,r2_std
0,manual,NaN,NaN,NaN,NaN,NaN,NaN
1,ridge,3.112272,0.555634,4.831318,1.106512,0.236368,0.499454
2,lasso,2.781482,0.583450,6.162875,2.047277,-0.018176,0.025811
3,cart,1.840971,0.227376,2.727881,0.503353,0.624369,0.532205
4,svr,2.395113,0.560500,5.899646,2.048665,0.082479,0.043938
5,xgb,1.383459,0.169045,1.942688,0.300818,0.801752,0.286150
6,rf,1.442760,0.228531,2.102303,0.641385,0.765261,0.314335
7,ann,NaN,NaN,NaN,NaN,NaN,NaN
